# Import modules

In [13]:
import pandas as pd
import html

# data pre-processing

In [2]:
data = pd.read_csv('sample.csv')

In [3]:
data

,Tweet Id,Tweet URL,Tweet Posted Time (UTC),Tweet Content,Tweet Type,Client,Retweets Received,Likes Received,Tweet Location,Tweet Language,...,Name,Username,User Bio,Verified or Non-Verified,Profile URL,Protected or Non-protected,User Followers,User Following,User Account Creation Date,Impressions
0,"""1167429261210218497""",https://twitter.com/animalhealthEU/status/1167...,30 Aug 2019 13:30:00,Pets change our lives &amp; become a part of o...,Tweet,Twitter Ads Composer,0,4,Brussels,English,...,AnimalhealthEurope,animalhealthEU,AnimalhealthEurope represents manufacturers of...,Non-Verified,https://twitter.com/animalhealthEU,Non-Protected,3697,542,17 Dec 2012 09:14:15,7394
1,"""1167375334670557185""",https://twitter.com/PennyBrohnUK/status/116737...,30 Aug 2019 09:55:43,Another spot of our #morethanmedicine bus in #...,Tweet,Twitter Web App,0,5,"Pill, Bristol",English,...,Penny Brohn UK,PennyBrohnUK,We help people live well with the impact of ca...,Non-Verified,https://twitter.com/PennyBrohnUK,Non-Protected,3227,1571,15 Sep 2010 09:44:02,6454
2,"""1167237977615097861""",https://twitter.com/lordbyronaf/status/1167237...,30 Aug 2019 00:49:54,What a great team ⁦@HealthSourceOH⁩ ⁦@Local12⁩...,ReTweet,Twitter for Android,0,0,"Ohio, USA",English,...,Lord ByronAF,lordbyronaf,"It's easier to be who you are, than it is to b...",Non-Verified,https://twitter.com/lordbyronaf,Non-Protected,7808,8617,25 Jul 2012 15:43:47,0
3,"""1167236897078480898""",https://twitter.com/CountessDavis/status/11672...,30 Aug 2019 00:45:37,What a great team ⁦@HealthSourceOH⁩ ⁦@Local12⁩...,ReTweet,Twitter for Android,0,0,NaN,English,...,Lisa Countess davis,CountessDavis,I am named after @ElvisPresley daughter Lisa M...,Non-Verified,https://twitter.com/CountessDavis,Non-Protected,291,81,26 Jan 2017 18:21:42,0
4,"""1167228378191204353""",https://twitter.com/Local12/status/11672283781...,30 Aug 2019 00:11:46,What a great team ⁦@HealthSourceOH⁩ ⁦@Local12⁩...,ReTweet,TweetDeck,0,0,"Cincinnati, OH",English,...,Local 12/WKRC-TV,Local12,Local 12 is #Cincinnati's trusted source for b...,Verified,https://twitter.com/Local12,Non-Protected,198675,651,02 Sep 2008 20:09:44,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,"""1145652120034717696""",https://twitter.com/UKNOAH/status/114565212003...,01 Jul 2019 11:15:25,#MorethanMedicine What does this mean? 🤔\nBy p...,ReTweet,Twitter Web Client,0,0,NaN,English,...,NOAH,UKNOAH,We represent the UK animal medicines industry:...,Non-Verified,https://twitter.com/UKNOAH,Non-Protected,2952,973,19 Dec 2011 16:22:28,0
382,"""1145637718493290497""",https://twitter.com/EMOverEasy/status/11456377...,01 Jul 2019 10:18:11,After a great weekend at #rebellion19 we are b...,Tweet,Twitter Web App,4,11,"DK Diner, USA",English,...,EMOverEasy,EMOverEasy,Tackling real world problems from the EM persp...,Non-Verified,https://twitter.com/EMOverEasy,Non-Protected,1345,361,09 Mar 2016 15:04:58,2694
383,"""1145628612739174400""",https://twitter.com/SIMV_RP/status/11456286127...,01 Jul 2019 09:42:00,#MorethanMedicine What does this mean? 🤔\nBy p...,ReTweet,Buffer,0,0,Paris,English,...,SIMV,SIMV_RP,Syndicat de l'Industrie du Médicament et réact...,Non-Verified,https://twitter.com/SIMV_RP,Non-Protected,948,387,27 Mar 2012 14:53:07,0
384,"""1145621305825341440""",https://twitter.com/wameyokw/status/1145621305...,01 Jul 2019 09:12:58,#MorethanMedicine What does this mean? 🤔\nBy p...,ReTweet,Twitter for Android,0,0,Nairobi Kenya,English,...,Kenneth Wameyo 🇰🇪 BVM,wameyokw,"Kenya Veterinary Association, Vet Kenya Google...",Non-Verified,https://twitter.com/wameyokw,Non-Protected,14771,5562,26 Apr 2009 09:35:59,0


In [9]:
tweets = data['Tweet Content']
tweets

0      Pets change our lives &amp; become a part of o...
1      Another spot of our #morethanmedicine bus in #...
2      What a great team ⁦@HealthSourceOH⁩ ⁦@Local12⁩...
3      What a great team ⁦@HealthSourceOH⁩ ⁦@Local12⁩...
4      What a great team ⁦@HealthSourceOH⁩ ⁦@Local12⁩...
                             ...                        
381    #MorethanMedicine What does this mean? 🤔\nBy p...
382    After a great weekend at #rebellion19 we are b...
383    #MorethanMedicine What does this mean? 🤔\nBy p...
384    #MorethanMedicine What does this mean? 🤔\nBy p...
385    #MorethanMedicine What does this mean? 🤔\nBy p...
Name: Tweet Content, Length: 386, dtype: object

In [10]:
file = open('sample.txt', 'w')
for tweet in tweets:
    file.write(tweet)

# Decode HTML Entities

So, far we already got the tweets but there are encoded html entities inside the tweets and we need to decode them first.